# Splitting Train and Test Sets based on clusters

The motive behind this experiment is to make sure our predictive model is not biased to the chemicals in the training set. So, what we will do here is use a list of chemicals clustered based on their similarities, find their neighbors, calculate their weight over the overall graph then use these weights to seperate training and test sets, in which a cluster of chemcials should only be in one of the sets

### Create dataframe from csv file

In [38]:
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit

In [6]:
clustered_chemicals = pd.read_csv("/home/raldisi/SE_KGE/resources/Clustered_chemicals.csv")

### Splitting training and testing sets based on clusters

In [6]:
train_inds, test_inds = next(GroupShuffleSplit(test_size=.20, n_splits=2, random_state = 7).split(clustered_chemicals, groups=clustered_chemicals['Cluster']))

train = clustered_chemicals.iloc[train_inds]
test = clustered_chemicals.iloc[test_inds]

### Splitting training and testing sets based on weighted clusters

In [1]:
import pandas as pd
import pybel
import networkx as nx
from tqdm import tqdm_notebook as tqdm

In [3]:
full_graph = pybel.from_pickle("/home/raldisi/SE_KGE/resources/fullgraph_without_sim.pickle")

In [7]:
clusters_dict = {i : clustered_chemicals['PubchemID'].loc[clustered_chemicals['Cluster'] == i].tolist()
                for i in range(1,clustered_chemicals.Cluster.nunique()+1)}

In this step, we make lists of the chemicals in each clusters and their neighbors

In [8]:
subgraphs_dict = {}
for cluster, chemicals in tqdm(clusters_dict.items()):
    chemicals_subgraph = []
    for chemical in chemicals:
        #formate chemical to BEL to match the nodes in full graph
        chemical = pybel.dsl.Abundance(namespace='pubchem', name=str(chemical)) 
        #ignore chemicals not in the graph
        if chemical not in full_graph.nodes():
            continue
        chemicals_subgraph.append(chemical)
        for neighbor in full_graph.neighbors(chemical):
            chemicals_subgraph.append(neighbor)
        #ignore empty lists
        if not chemicals_subgraph:
            continue
    subgraphs_dict[cluster] = list(dict.fromkeys(chemicals_subgraph)) # to remove duplicates

Now, we calculate the weights of each cluster by counting the number of edges in the subgraph and dividing it by the number of edges in the fullgraph

In [9]:
fullgraph_edges = len(full_graph.edges())
cluster_weights = {}
for cluster, nodes in tqdm(subgraphs_dict.items()):
    subgraph = full_graph.subgraph(nodes)
    edges = len(subgraph.edges())
    cluster_weights[cluster] = edges/fullgraph_edges

Next we create a csv file from the previous clusters file that contains the chemical, its cluster and its weight

In [20]:
clustered_chemicals['weight'] = None

In [24]:
for cluster, weight in cluster_weights.items():
    clustered_chemicals.loc[clustered_chemicals['Cluster'] == cluster, 'weight'] = weight

In [26]:
clustered_chemicals.to_csv('/home/raldisi/SE_KGE/resources/clusters_weights.csv')

Create an weighted edgelist

In [11]:
weights_df = pd.read_csv('/home/raldisi/SE_KGE/resources/clusters_weights.csv', index_col=False, dtype={'PubchemID': str})

In [12]:
weights = {}
for index, row in weights_df.iterrows():
    weights[pybel.dsl.Abundance(namespace='pubchem', name=row['PubchemID'])] = row['weight']

In [20]:
mapping_df = pd.read_csv('/home/raldisi/SE_KGE/resources/fullgraph_nodes_mapping.tsv', index_col=False, dtype={'NodeName': str, 'NodeNamespace':str})

In [21]:
mapping_df.head()

,NodeID,NodeNamespace,NodeName
0,1,pubchem,85
1,2,umls,Abdominal cramps
2,3,umls,Abdominal pain
3,4,umls,Gastrointestinal pain
4,5,umls,Amblyopia


In [30]:
mapping_dict={}
for index, row in mapping_df.iterrows():
    if row['NodeNamespace'] == 'pubchem':
        mapping_dict[pybel.dsl.Abundance(namespace=row['NodeNamespace'], name=row['NodeName'])] = row['NodeID']
    elif row['NodeNamespace'] == 'umls':
        mapping_dict[pybel.dsl.Pathology(namespace=row['NodeNamespace'], name=row['NodeName'])] = row['NodeID']
    else:
        mapping_dict[pybel.dsl.Protein(namespace=row['NodeNamespace'], name=row['NodeName'])] = row['NodeID']


In [32]:
with open('/home/raldisi/SE_KGE/resources/fullgraph_weighted.edgelist','w') as outputfile:
    outputfile.write('Source\tTarget\tWeight\n')
    for source, target in full_graph.edges():
        if source not in weights:
            outputfile.write('%s\t%s\t%f\n' %(mapping_dict[source], mapping_dict[target], 0.0))
        else:
            outputfile.write('%s\t%s\t%f\n' %(mapping_dict[source], mapping_dict[target], weights[source]))

In [40]:
weighted_edgelist = pd.read_csv('/home/raldisi/SE_KGE/resources/fullgraph_weighted.edgelist', sep='\t')

Split training and testing sets based on weights (each weight corresponds to a cluster)

In [45]:
train_inds, test_inds = next(GroupShuffleSplit(test_size=.20, n_splits=2, random_state = 7).split(weighted_edgelist, groups=weighted_edgelist['Weight']))

training = weighted_edgelist.iloc[train_inds]
testing = weighted_edgelist.iloc[test_inds]

In [52]:
with open('training_edgelist.txt','w') as trainingfile:
    for index, row in training.iterrows():
        trainingfile.write('%d\t%d\n' % (row['Source'], row['Target']))

In [53]:
with open('testing_edgelist.txt','w') as trainingfile:
    for index, row in testing.iterrows():
        trainingfile.write('%d\t%d\n' % (row['Source'], row['Target']))

Next steps:
- add this splitting code to BioNEV code to get results